In [1]:
import numpy as np
import pandas as pd
from sklearn import preprocessing
import math

In [2]:
le = preprocessing.LabelEncoder()

In [5]:
data = pd.read_csv("healthcare-dataset-stroke-data.csv")

In [6]:
data = data.drop(labels="id",axis=1)

In [7]:
data.columns

Index(['gender', 'age', 'hypertension', 'heart_disease', 'ever_married',
       'work_type', 'Residence_type', 'avg_glucose_level', 'bmi',
       'smoking_status', 'stroke'],
      dtype='object')

In [8]:
D = data.copy()

In [9]:
D.head()

,gender,age,hypertension,heart_disease,ever_married,work_type,Residence_type,avg_glucose_level,bmi,smoking_status,stroke
0,Male,67.0,0,1,Yes,Private,Urban,228.69,36.6,formerly smoked,1
1,Female,61.0,0,0,Yes,Self-employed,Rural,202.21,NaN,never smoked,1
2,Male,80.0,0,1,Yes,Private,Rural,105.92,32.5,never smoked,1
3,Female,49.0,0,0,Yes,Private,Urban,171.23,34.4,smokes,1
4,Female,79.0,1,0,Yes,Self-employed,Rural,174.12,24.0,never smoked,1


In [10]:
X = data[["gender","age","hypertension","heart_disease","ever_married","work_type","Residence_type","avg_glucose_level","bmi","smoking_status"]]
X = X.T.values
# to store the gender, ever_married, work_type, Residence_type, smoking_status
invTransform = []

aa = le.fit_transform(X[0])
invTransform.append(le.inverse_transform(aa))
D['gender']=aa

aa = le.fit_transform(X[4])
invTransform.append(le.inverse_transform(aa))
D["ever_married"] = aa

aa = le.fit_transform(X[5])
invTransform.append(le.inverse_transform(aa))
D['work_type'] = aa

aa = le.fit_transform(X[6])
invTransform.append(le.inverse_transform(aa))
D['Residence_type'] = aa

aa = le.fit_transform(X[9])
invTransform.append(le.inverse_transform(aa))
D['smoking_status'] = aa

In [11]:
D.head()

,gender,age,hypertension,heart_disease,ever_married,work_type,Residence_type,avg_glucose_level,bmi,smoking_status,stroke
0,1,67.0,0,1,1,2,1,228.69,36.6,1,1
1,0,61.0,0,0,1,3,0,202.21,NaN,2,1
2,1,80.0,0,1,1,2,0,105.92,32.5,2,1
3,0,49.0,0,0,1,2,1,171.23,34.4,3,1
4,0,79.0,1,0,1,3,0,174.12,24.0,2,1


In [12]:
D.fillna(value = D['bmi'].median(), inplace = True)

In [15]:
d = D[D['gender'] == 'Other'].index
D.drop(d, inplace = True)

In [16]:
x = D.drop('stroke', axis = 1)
y = D['stroke']

In [17]:
from imblearn.over_sampling import RandomOverSampler

In [19]:
os = RandomOverSampler(sampling_strategy = 1)
x_ros, y_ros = os.fit_resample(x, y)
x = pd.DataFrame(x_ros)
y = pd.DataFrame(y_ros)

In [24]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.metrics import classification_report
from sklearn.preprocessing import StandardScaler

import warnings
warnings.filterwarnings("ignore")

In [25]:
x_train, x_test, y_train, y_test = train_test_split(x, y, train_size = 0.75, shuffle = True, random_state = 1)
scaler = StandardScaler()
scaler.fit(x_train)
x_train = pd.DataFrame(scaler.fit_transform(x_train), index = x_train.index, columns = x_train.columns)
x_test = pd.DataFrame(scaler.fit_transform(x_test), index = x_test.index, columns = x_test.columns)

In [27]:
knn = KNeighborsClassifier(n_neighbors = 2)
knn.fit(x_train, y_train)
knn_predicted = knn.predict(x_test)
knn_conf_matrix = confusion_matrix(y_test, knn_predicted)
knn_acc_score = accuracy_score(y_test, knn_predicted)

In [30]:
print("Accuracy of K-NeighborsClassifier:",knn_acc_score*100,'\n')
print(classification_report(y_test, knn_predicted))

Accuracy of K-NeighborsClassifier: 97.49074454956808 

              precision    recall  f1-score   support

           0       0.99      0.96      0.97      1200
           1       0.96      0.99      0.98      1231

    accuracy                           0.97      2431
   macro avg       0.98      0.97      0.97      2431
weighted avg       0.98      0.97      0.97      2431

